In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]=""

In [11]:
%load_ext autoreload

In [12]:
import pandas as pd
from tqdm import tqdm
import torch
import sys
import time
import datetime
from itertools import product
import os
import numpy as np
from sklearn import tree
from sklearn import linear_model
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, f1_score
import matplotlib.pyplot as plt
from functools import partial
import shelve
import tqdm
import pickle
from collections import Counter

In [13]:
from ogb.graphproppred import Evaluator
from tdc.benchmark_group import admet_group

In [14]:
%autoreload 2
from huggingmolecules import MatConfig, MatFeaturizer, MatModel
from huggingmolecules import GroverConfig, GroverFeaturizer, GroverModel
from huggingmolecules import RMatConfig, RMatFeaturizer, RMatModel
from models import StrippedMatModel, StrippedGroverModel, StrippedRMatModel

In [15]:
%autoreload 2
from utils import *
from datasets import *
from predict_procedure import *

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Error: Unable to import pysam. Please make sure it is installed.
Error: Unable to import pysam. Please make sure it is installed.


Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [16]:
# from mat import MolecularAttentionTransformer

# model = MolecularAttentionTransformer()
# model.predict(["CN1CC[C@]23C(=O)C[C@@H]4C(=CCO[C@H]5CC(=O)N(c6ccccc62)[C@@H]3[C@@H]54)C1"]) ## tooks 11m 4s (!)

In [7]:
# group = admet_group(path = 'data/')
# benchmark = group.get("CYP2D6_Substrate_CarbonMangels")

# dataset = benchmark["test"]

# print(dataset[dataset.Drug == "CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C"].to_markdown())

# Output:
# |     | smiles                             |   Y |
# |----:|:-----------------------------------|----:|
# |  58 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   1 |
# | 226 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   0 |else 'regression'

In [8]:
# import pathlib

# calculated = set(map(str, pathlib.Path("embeddings/").glob("*/*.zip")))

# pairs = product(get_dataset_names(), get_embedder_names())

# with open("embedding_params.txt", "w") as f:
#     for dataset_name, model_name in pairs:
#         if f"embeddings/{dataset_name}/{model_name}.zip" not in calculated:
#             print(dataset_name, model_name, file=f)


In [18]:
group = admet_group(path='data/')
benchmark = group.get("CYP2C9_Substrate_CarbonMangels")

predictions = {}
name = benchmark['name']
a = benchmark["test"]

Found local copy...


In [22]:
a.Y.sum()

38

In [45]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = list(product(get_classification_names(), get_embedder_names()))
calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

params = list(product(list(set(pairs) & set(calculated)), ["ridge","forest"]))


with open("predict_params.txt", "w") as f:
    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" not in done:
            print(dataset_name, model_name, head_model, file=f)

In [22]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = list(product(get_dataset_names(), get_embedder_names()))
calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

params = list(product(list(set(pairs) & set(calculated)), ["ridge","forest"]))


with open("predict_params.txt", "w") as f:
    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" in done and dataset_name in get_classification_names():
            print(dataset_name, model_name, head_model, file=f)

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifierCV, RidgeCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve, auc
from scipy.stats import spearmanr

In [18]:
dataset_name, base_model, head_model = "CYP2C9_Substrate_CarbonMangels", "ChemBERTa-10M-MTR", "forest"

_, split_idx = get_dataset(dataset_name)
df = pd.read_pickle(f"embeddings/{dataset_name}/{base_model}.zip").drop("features", axis=1, errors="ignore")
df

predict_fun = get_predict_fun(dataset_name, head_model)

scoring, predict_proba = get_metric(dataset_name)

preds = predict_fun(df, split_idx, scoring=scoring, predict_proba=predict_proba)
preds

evaluate(dataset_name, preds)

# procedure(dataset_name, base_model, head_model)


Found local copy...
Loading...


Done!
Found local copy...
Found local copy...


('pr-auc', 0.451)

In [14]:
preds

array([[0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [35]:
evaluate(dataset_name, sigmoid(preds))

Found local copy...


('pr-auc', 0.291)

In [36]:
preds

array([[0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [26]:
(preds > 0.5) * 1

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [21]:
import math

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

sigmoid(preds - 0.5)

array([0.42161421, 0.44094688, 0.42674479, 0.52222318, 0.44242793,
       0.39327176, 0.46599716, 0.49025719, 0.45238108, 0.39434831,
       0.44407448, 0.43001179, 0.4744063 , 0.50158968, 0.41972731,
       0.47448883, 0.41253184, 0.4187567 , 0.4239842 , 0.43290081,
       0.50124254, 0.46611831, 0.4480864 , 0.43627427, 0.46080884,
       0.41171397, 0.48890338, 0.47207135, 0.4467991 , 0.41330773,
       0.45965931, 0.41552689, 0.47110107, 0.41650123, 0.45910426,
       0.421469  , 0.38674862, 0.41238064, 0.44726588, 0.43835653,
       0.42135996, 0.40725152, 0.50177111, 0.50876111, 0.46087121,
       0.40818538, 0.40765447, 0.42827417, 0.42569647, 0.42939636,
       0.45338923, 0.47562187, 0.40627918, 0.43619709, 0.45550516,
       0.44841165, 0.45032407, 0.48074109, 0.43216175, 0.40175156,
       0.41086103, 0.38942793, 0.39464983, 0.49578446, 0.39557991,
       0.39120213, 0.48614281, 0.47539514, 0.48071923, 0.48807791,
       0.47821058, 0.47872144, 0.48521299, 0.47132218, 0.44999

In [70]:
with open(f"results/ogbg-molfreesolv_mat_masking_2M_forest.pkl", "rb") as f:
    tmp = pickle.load(f)
tmp

EOFError: Ran out of input

In [58]:
for dataset_name in get_dataset_names():
    df = pd.read_pickle(f"embeddings/{dataset_name}/mol2vec.zip")
    print(dataset_name, len(df))

ogbg-molesol 1128
ogbg-molfreesolv 642
ogbg-mollipo 4200
ogbg-molbace 1513
ogbg-molbbbp 2039
ogbg-molclintox 1477
ogbg-molsider 1427
ogbg-moltox21 7831
ogbg-moltoxcast 8576
ogbg-molmuv 93087
ogbg-molhiv 41127
Caco2_Wang 910
Half_Life_Obach 667
Lipophilicity_AstraZeneca 4200
Solubility_AqSolDB 9982
PPBR_AZ 1614
VDss_Lombardo 1130
Clearance_Hepatocyte_AZ 1213
HIA_Hou 578
Pgp_Broccatelli 1218
Bioavailability_Ma 640
BBB_Martins 2030
CYP2D6_Veith 13130
CYP3A4_Veith 12328
CYP2C9_Veith 12092
CYP2C9_Substrate_CarbonMangels 669
CYP2D6_Substrate_CarbonMangels 667
CYP3A4_Substrate_CarbonMangels 670


In [87]:
read(f"{dataset_name}_{base_model}_ridge.pkl")

{'dataset': 'Caco2_Wang',
 'base_model': 'ChemBERTa-5M-MTR',
 'head_model': 'rigid',
 'metric': 'mae',
 'result': 0.333}

In [89]:
group = admet_group(path='data/')
group.dataset_names

Found local copy...


['caco2_wang',
 'hia_hou',
 'pgp_broccatelli',
 'bioavailability_ma',
 'lipophilicity_astrazeneca',
 'solubility_aqsoldb',
 'bbb_martins',
 'ppbr_az',
 'vdss_lombardo',
 'cyp2d6_veith',
 'cyp3a4_veith',
 'cyp2c9_veith',
 'cyp2d6_substrate_carbonmangels',
 'cyp3a4_substrate_carbonmangels',
 'cyp2c9_substrate_carbonmangels',
 'half_life_obach',
 'clearance_microsome_az',
 'clearance_hepatocyte_az',
 'herg',
 'ames',
 'dili',
 'ld50_zhu']